In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Import Data

In [2]:
Data = pd.read_csv('AmesHousing.csv')

In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

In [4]:
Data.head()

Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
3      4  526353030           20        RL          93.0     11160   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   

  Alley Lot Shape Land Contour  ... Pool Area Pool QC  Fence Misc Feature  \
0   NaN       IR1          Lvl  ...         0     NaN    NaN          NaN   
1   NaN       Reg          Lvl  ...         0     NaN  MnPrv          NaN   
2   NaN       IR1          Lvl  ...         0     NaN    NaN         Gar2   
3   NaN       Reg          Lvl  ...         0     NaN    NaN          NaN   
4   NaN       IR1          Lvl  ...         0     NaN  MnPrv          NaN   

  Misc Val Mo Sold Yr Sold Sale Type  Sale Condition  SalePrice  
0        0       5    2010       WD           Normal     215000  
1        0       6    2010       WD           Normal     105000  
2    12500       6    2010       WD           Normal     172000  
3        0       4    2010       WD           Normal     244000  
4        0       3    2010       WD           Normal     189900  

[5 rows x 82 columns]

In [5]:
Data.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      490
                 ... 
Mo Sold             0
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
Length: 82, dtype: int64

# Fill missing values for numeric columns

In [6]:
numeric_columns = Data.select_dtypes(include=[np.number]).columns
Data[numeric_columns] = Data[numeric_columns].fillna(Data[numeric_columns].mean())

In [7]:
Data = pd.get_dummies(Data , drop_first=True)

In [8]:
X = Data.drop('SalePrice', axis=1)
Y = Data['SalePrice']

# Normalize features and target variable

In [9]:
# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

# Normalize target variable
scaler_Y = StandardScaler()
Y = Y.values.reshape(-1, 1)
Y_scaled = scaler_Y.fit_transform(Y).ravel()

# check for Outliers

In [18]:
# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outliers = iso_forest.fit_predict(X)
outliers = outliers == -1  # Outliers are labeled as -1

plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 7], c=outliers, cmap='coolwarm', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 7')
plt.title('Isolation Forest Outliers')
plt.colorbar(label='Outlier')
plt.show()

In [19]:
# The Red color show the outlier in the data
# The above graph is noly showing the graph of feature 1 and 7
# Remove outliers
X_cleaned = X[~outliers]
Y_cleaned = Y_scaled[~outliers]

In [12]:
X_train,  X_test, y_train, y_test = train_test_split(X_cleaned, Y_cleaned, test_size=0.2, random_state = 42)

In [21]:
# Define the model
# the reason for choosing SGD_regression is that we can give control the learning rate , iteration and other
# important parameters
sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42 , eta0=0.001)

# Train the model
sgd_regressor.fit(X_train, y_train)

SGDRegressor(eta0=0.001, random_state=42)

In [14]:
y_pred = sgd_regressor.predict(X_test)

In [15]:
mse =  mean_squared_error(y_test, y_pred)
r2 =  r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.08554801158255006
R-squared: 0.9128414176542377


# Congrulation you have 91.3% accuracy :)